# Nagpur City - Historical AQI Data Analysis & Visualization Dashboard

## 📊 Objective
Comprehensive analysis of Nagpur's Air Quality Index (AQI) data from RO Office dataset to support environmental decision-making and policy recommendations.

**Dataset:** RO Office.xlsx  
**Location:** Nagpur City  
**Parameters:** SO2, NOx, RSPM, and AQI  
**Analysis Date:** November 9, 2025

## Step 1: Import Required Libraries

In [1]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Set visualization styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Step 2: Load and Initial Data Exploration

In [2]:
# Load the dataset
df = pd.read_excel('RO Office.xlsx')

print("="*70)
print("📁 DATASET OVERVIEW")
print("="*70)
print(f"Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n📅 Date Range: {df['Date'].min()} to {df['Date'].max()}")
print(f"⏱️  Time Span: {(df['Date'].max() - df['Date'].min()).days} days (~{(df['Date'].max() - df['Date'].min()).days/365:.1f} years)")

print("\n" + "="*70)
print("📋 COLUMN INFORMATION")
print("="*70)
print(df.columns.tolist())

print("\n" + "="*70)
print("👀 FIRST 10 ROWS")
print("="*70)
df.head(10)

📁 DATASET OVERVIEW
Dataset Shape: 2974 rows × 5 columns

📅 Date Range: 2009-07-27 00:00:00 to 2019-07-18 00:00:00
⏱️  Time Span: 3643 days (~10.0 years)

📋 COLUMN INFORMATION
['Date', 'SO2 ', 'Nitrogen Oxide (Nox)\nµg/m3', 'Respirable Suspended Particulate Matter(RSPM)\nµg/m3', 'AQI']

👀 FIRST 10 ROWS


,Date,SO2,Nitrogen Oxide (Nox)\nµg/m3,Respirable Suspended Particulate Matter(RSPM)\nµg/m3,AQI
0,2009-07-27,9,29,42.0,42.0
1,2009-07-28,8,27,47.0,47.0
2,2009-07-29,8,30,48.0,48.0
3,2009-07-30,9,28,53.0,53.0
4,2009-07-31,9,27,59.0,59.0
5,2009-08-06,9,27,52.0,52.0
6,2009-08-07,9,30,63.0,63.0
7,2009-08-08,9,27,44.0,44.0
8,2009-08-10,9,24,46.0,46.0
9,2009-08-13,9,31,49.0,49.0


## Step 3: Data Cleaning and Preprocessing

In [3]:
# Clean column names
df.columns = ['Date', 'SO2', 'NOx', 'RSPM', 'AQI']

# Convert columns to numeric (handling any non-numeric values)
df['SO2'] = pd.to_numeric(df['SO2'], errors='coerce')
df['NOx'] = pd.to_numeric(df['NOx'], errors='coerce')
df['RSPM'] = pd.to_numeric(df['RSPM'], errors='coerce')
df['AQI'] = pd.to_numeric(df['AQI'], errors='coerce')

# Extract temporal features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month_Name'] = df['Date'].dt.strftime('%B')
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()
df['Quarter'] = df['Date'].dt.quarter
df['Season'] = df['Month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Monsoon', 10: 'Monsoon', 11: 'Autumn'
})

# Create AQI categories based on Indian AQI standards
def categorize_aqi(aqi):
    if pd.isna(aqi):
        return 'Unknown'
    elif aqi <= 50:
        return 'Good'
    elif aqi <= 100:
        return 'Satisfactory'
    elif aqi <= 200:
        return 'Moderate'
    elif aqi <= 300:
        return 'Poor'
    elif aqi <= 400:
        return 'Very Poor'
    else:
        return 'Severe'

df['AQI_Category'] = df['AQI'].apply(categorize_aqi)

print("✅ Data cleaning completed!")
print(f"\n📊 Missing values:")
print(df[['SO2', 'NOx', 'RSPM', 'AQI']].isnull().sum())
print(f"\n📈 Data types:")
print(df.dtypes)

✅ Data cleaning completed!

📊 Missing values:
SO2     155
NOx     153
RSPM    152
AQI     243
dtype: int64

📈 Data types:
Date            datetime64[ns]
SO2                    float64
NOx                    float64
RSPM                   float64
AQI                    float64
Year                   float64
Month                  float64
Month_Name              object
Day                    float64
Weekday                 object
Quarter                float64
Season                  object
AQI_Category            object
dtype: object


## Step 4: Statistical Summary and Data Quality Assessment

In [4]:
# Statistical summary
print("="*70)
print("📊 STATISTICAL SUMMARY")
print("="*70)
print(df[['SO2', 'NOx', 'RSPM', 'AQI']].describe().round(2))

print("\n" + "="*70)
print("🎯 AQI CATEGORY DISTRIBUTION")
print("="*70)
category_counts = df['AQI_Category'].value_counts()
print(category_counts)
print(f"\nPercentage distribution:")
print((category_counts / len(df) * 100).round(2))

print("\n" + "="*70)
print("📅 YEARLY DATA COVERAGE")
print("="*70)
yearly_coverage = df.groupby('Year').size()
print(yearly_coverage)

📊 STATISTICAL SUMMARY
           SO2      NOx     RSPM      AQI
count  2819.00  2821.00  2822.00  2731.00
mean      9.45    28.95    45.75    62.42
std       3.89    11.87    48.06    21.84
min     -87.00  -398.00  -216.00     0.00
25%       8.00    23.00    46.00    49.00
50%       9.00    28.00    55.00    58.00
75%      11.00    35.00    66.00    71.00
max      43.00    69.00    99.00   398.00

🎯 AQI CATEGORY DISTRIBUTION
AQI_Category
Satisfactory    1795
Good             750
Unknown          243
Moderate         185
Very Poor          1
Name: count, dtype: int64

Percentage distribution:
AQI_Category
Satisfactory    60.36
Good            25.22
Unknown          8.17
Moderate         6.22
Very Poor        0.03
Name: count, dtype: float64

📅 YEARLY DATA COVERAGE
Year
2009.0    110
2010.0    285
2011.0    233
2012.0    263
2013.0    288
2014.0    292
2015.0    299
2016.0    308
2017.0    291
2018.0    309
2019.0    150
dtype: int64


## Step 5: Time Series Visualization - Historical AQI Trends

In [5]:
# Complete historical AQI trend with category bands
fig = go.Figure()

# Add AQI line
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['AQI'],
    mode='lines',
    name='AQI',
    line=dict(color='#2E86AB', width=1.5),
    hovertemplate='<b>Date:</b> %{x}<br><b>AQI:</b> %{y:.1f}<extra></extra>'
))

# Add category bands
fig.add_hrect(y0=0, y1=50, fillcolor="green", opacity=0.1, line_width=0, annotation_text="Good", annotation_position="right")
fig.add_hrect(y0=50, y1=100, fillcolor="yellow", opacity=0.1, line_width=0, annotation_text="Satisfactory", annotation_position="right")
fig.add_hrect(y0=100, y1=200, fillcolor="orange", opacity=0.1, line_width=0, annotation_text="Moderate", annotation_position="right")
fig.add_hrect(y0=200, y1=300, fillcolor="red", opacity=0.1, line_width=0, annotation_text="Poor", annotation_position="right")
fig.add_hrect(y0=300, y1=400, fillcolor="purple", opacity=0.1, line_width=0, annotation_text="Very Poor", annotation_position="right")

fig.update_layout(
    title='<b>Nagpur AQI Historical Trends (Complete Timeline)</b>',
    xaxis_title='Date',
    yaxis_title='Air Quality Index (AQI)',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

## Step 6: Multi-Pollutant Time Series Analysis

In [6]:
# All pollutants over time
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['SO2'], name='SO2', line=dict(color='#E63946')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['NOx'], name='NOx', line=dict(color='#F77F00')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['RSPM'], name='RSPM', line=dict(color='#06D6A0')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['AQI'], name='AQI', line=dict(color='#118AB2', width=2)))

fig.update_layout(
    title='<b>Multi-Pollutant Trends: SO2, NOx, RSPM & AQI</b>',
    xaxis_title='Date',
    yaxis_title='Concentration (µg/m³)',
    height=500,
    hovermode='x unified',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)')
)

fig.show()

## Step 7: Yearly Trend Analysis

In [7]:
# Yearly average trends
yearly_avg = df.groupby('Year')[['SO2', 'NOx', 'RSPM', 'AQI']].mean().reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(x=yearly_avg['Year'], y=yearly_avg['SO2'], name='SO2', marker_color='#E63946'))
fig.add_trace(go.Bar(x=yearly_avg['Year'], y=yearly_avg['NOx'], name='NOx', marker_color='#F77F00'))
fig.add_trace(go.Bar(x=yearly_avg['Year'], y=yearly_avg['RSPM'], name='RSPM', marker_color='#06D6A0'))
fig.add_trace(go.Scatter(x=yearly_avg['Year'], y=yearly_avg['AQI'], name='AQI (Trend)', 
                         mode='lines+markers', line=dict(color='#118AB2', width=3),
                         marker=dict(size=8)))

fig.update_layout(
    title='<b>Year-over-Year Pollutant & AQI Trends</b>',
    xaxis_title='Year',
    yaxis_title='Average Concentration (µg/m³)',
    barmode='group',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

## Step 8: Seasonal Pattern Analysis

In [8]:
# Seasonal analysis
season_order = ['Winter', 'Spring', 'Summer', 'Monsoon', 'Autumn']
seasonal_avg = df.groupby('Season')[['SO2', 'NOx', 'RSPM', 'AQI']].mean().reindex(season_order)

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=seasonal_avg['SO2'].values,
    theta=season_order,
    fill='toself',
    name='SO2',
    line_color='#E63946'
))

fig.add_trace(go.Scatterpolar(
    r=seasonal_avg['NOx'].values,
    theta=season_order,
    fill='toself',
    name='NOx',
    line_color='#F77F00'
))

fig.add_trace(go.Scatterpolar(
    r=seasonal_avg['RSPM'].values,
    theta=season_order,
    fill='toself',
    name='RSPM',
    line_color='#06D6A0'
))

fig.add_trace(go.Scatterpolar(
    r=seasonal_avg['AQI'].values,
    theta=season_order,
    fill='toself',
    name='AQI',
    line_color='#118AB2'
))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, seasonal_avg.max().max()])),
    title='<b>Seasonal Variation in Air Quality Parameters</b>',
    height=550,
    showlegend=True
)

fig.show()

## Step 9: Monthly Patterns & Heatmap

In [9]:
# Create year-month heatmap
heatmap_data = df.pivot_table(values='AQI', index='Year', columns='Month', aggfunc='mean')

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    y=heatmap_data.index,
    colorscale='RdYlGn_r',
    text=heatmap_data.values.round(1),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="AQI")
))

fig.update_layout(
    title='<b>AQI Heatmap: Year × Month Pattern</b>',
    xaxis_title='Month',
    yaxis_title='Year',
    height=600,
    template='plotly_white'
)

fig.show()

## Step 10: AQI Category Distribution

In [10]:
# AQI category distribution
category_order = ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']
category_colors = ['#00E400', '#FFFF00', '#FF7E00', '#FF0000', '#8F3F97', '#7E0023']

category_counts = df['AQI_Category'].value_counts().reindex(category_order, fill_value=0)

fig = go.Figure()

fig.add_trace(go.Pie(
    labels=category_counts.index,
    values=category_counts.values,
    marker=dict(colors=category_colors),
    textinfo='label+percent+value',
    hole=0.4
))

fig.update_layout(
    title='<b>Overall AQI Category Distribution</b>',
    height=500,
    annotations=[dict(text='AQI<br>Categories', x=0.5, y=0.5, font_size=14, showarrow=False)]
)

fig.show()

## Step 11: Pollutant Distribution Analysis

In [11]:
# Box plots for pollutant distribution
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('SO2 Distribution', 'NOx Distribution', 'RSPM Distribution', 'AQI Distribution')
)

fig.add_trace(go.Box(y=df['SO2'], name='SO2', marker_color='#E63946'), row=1, col=1)
fig.add_trace(go.Box(y=df['NOx'], name='NOx', marker_color='#F77F00'), row=1, col=2)
fig.add_trace(go.Box(y=df['RSPM'], name='RSPM', marker_color='#06D6A0'), row=2, col=1)
fig.add_trace(go.Box(y=df['AQI'], name='AQI', marker_color='#118AB2'), row=2, col=2)

fig.update_layout(
    title='<b>Pollutant Distribution (Box Plots)</b>',
    height=700,
    showlegend=False,
    template='plotly_white'
)

fig.show()

## Step 12: Correlation Analysis

In [12]:
# Correlation heatmap
corr_matrix = df[['SO2', 'NOx', 'RSPM', 'AQI']].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values.round(3),
    texttemplate='%{text}',
    textfont={"size": 14},
    colorbar=dict(title="Correlation")
))

fig.update_layout(
    title='<b>Pollutant Correlation Matrix</b>',
    height=500,
    template='plotly_white'
)

fig.show()

print("\n" + "="*70)
print("🔗 CORRELATION INSIGHTS")
print("="*70)
print(corr_matrix.round(3))


🔗 CORRELATION INSIGHTS
        SO2    NOx   RSPM    AQI
SO2   1.000  0.328 -0.122  0.284
NOx   0.328  1.000 -0.129  0.137
RSPM -0.122 -0.129  1.000 -0.445
AQI   0.284  0.137 -0.445  1.000


## Step 13: Comparative Analysis - Best vs Worst Years

In [13]:
# Find best and worst years
yearly_aqi = df.groupby('Year')['AQI'].mean().sort_values()
best_year = yearly_aqi.index[0]
worst_year = yearly_aqi.index[-1]

print(f"🏆 Best Air Quality Year: {best_year} (Avg AQI: {yearly_aqi[best_year]:.2f})")
print(f"⚠️  Worst Air Quality Year: {worst_year} (Avg AQI: {yearly_aqi[worst_year]:.2f})")

# Create comparison chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=yearly_aqi.index,
    y=yearly_aqi.values,
    marker=dict(
        color=yearly_aqi.values,
        colorscale='RdYlGn_r',
        showscale=True,
        colorbar=dict(title="Avg AQI")
    ),
    text=yearly_aqi.values.round(1),
    textposition='outside',
    hovertemplate='<b>Year:</b> %{x}<br><b>Avg AQI:</b> %{y:.2f}<extra></extra>'
))

fig.update_layout(
    title='<b>Average AQI by Year - Comparative Analysis</b>',
    xaxis_title='Year',
    yaxis_title='Average AQI',
    height=500,
    template='plotly_white'
)

fig.show()

🏆 Best Air Quality Year: 2016.0 (Avg AQI: 53.88)
⚠️  Worst Air Quality Year: 2010.0 (Avg AQI: 73.76)


## Step 14: Exceedance Analysis - Days Above Standards

In [14]:
# Calculate exceedance days (AQI > 100 = Moderate or worse)
df['Exceeds_Standard'] = df['AQI'] > 100

exceedance_yearly = df.groupby('Year').agg({
    'Exceeds_Standard': 'sum',
    'AQI': 'count'
}).reset_index()
exceedance_yearly.columns = ['Year', 'Exceedance_Days', 'Total_Days']
exceedance_yearly['Exceedance_Percentage'] = (exceedance_yearly['Exceedance_Days'] / exceedance_yearly['Total_Days'] * 100)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=exceedance_yearly['Year'], y=exceedance_yearly['Exceedance_Days'], 
           name='Days Above Standard', marker_color='#E63946'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=exceedance_yearly['Year'], y=exceedance_yearly['Exceedance_Percentage'], 
               name='% of Days', mode='lines+markers', line=dict(color='#118AB2', width=3),
               marker=dict(size=8)),
    secondary_y=True
)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="<b>Number of Days</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Percentage (%)</b>", secondary_y=True)

fig.update_layout(
    title='<b>Annual Exceedance Analysis (AQI > 100)</b>',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

print(f"\n⚠️ Total days with AQI > 100: {df['Exceeds_Standard'].sum()} out of {len(df)} days ({df['Exceeds_Standard'].sum()/len(df)*100:.1f}%)")


⚠️ Total days with AQI > 100: 186 out of 2974 days (6.3%)


## Step 15: Moving Average Trend Analysis

In [15]:
# Calculate moving averages
df_sorted = df.sort_values('Date')
df_sorted['AQI_MA_30'] = df_sorted['AQI'].rolling(window=30, min_periods=1).mean()
df_sorted['AQI_MA_90'] = df_sorted['AQI'].rolling(window=90, min_periods=1).mean()
df_sorted['AQI_MA_365'] = df_sorted['AQI'].rolling(window=365, min_periods=1).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_sorted['Date'], 
    y=df_sorted['AQI'], 
    mode='lines', 
    name='Daily AQI',
    line=dict(color='lightgray', width=1),
    opacity=0.5
))

fig.add_trace(go.Scatter(
    x=df_sorted['Date'], 
    y=df_sorted['AQI_MA_30'], 
    mode='lines', 
    name='30-Day MA',
    line=dict(color='#F77F00', width=2)
))

fig.add_trace(go.Scatter(
    x=df_sorted['Date'], 
    y=df_sorted['AQI_MA_90'], 
    mode='lines', 
    name='90-Day MA',
    line=dict(color='#06D6A0', width=2)
))

fig.add_trace(go.Scatter(
    x=df_sorted['Date'], 
    y=df_sorted['AQI_MA_365'], 
    mode='lines', 
    name='365-Day MA',
    line=dict(color='#118AB2', width=3)
))

fig.update_layout(
    title='<b>AQI Trend with Moving Averages (Smoothing)</b>',
    xaxis_title='Date',
    yaxis_title='AQI',
    height=550,
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

## Step 16: Decision-Making Insights & Recommendations

In [16]:
# Comprehensive insights for decision making
print("="*80)
print(" " * 20 + "📊 NAGPUR AIR QUALITY - DECISION INSIGHTS")
print("="*80)

# 1. Overall Statistics
print("\n1️⃣  OVERALL AIR QUALITY STATUS")
print("-" * 80)
print(f"   • Average AQI: {df['AQI'].mean():.2f}")
print(f"   • Median AQI: {df['AQI'].median():.2f}")
print(f"   • AQI Range: {df['AQI'].min():.2f} - {df['AQI'].max():.2f}")
print(f"   • Data Coverage: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")

# 2. Category Breakdown
print("\n2️⃣  AQI CATEGORY BREAKDOWN")
print("-" * 80)
for category in ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe']:
    count = (df['AQI_Category'] == category).sum()
    percent = count / len(df) * 100
    print(f"   • {category:15s}: {count:5d} days ({percent:5.2f}%)")

# 3. Seasonal Insights
print("\n3️⃣  SEASONAL PATTERNS")
print("-" * 80)
seasonal_aqi = df.groupby('Season')['AQI'].mean().sort_values(ascending=False)
for season, aqi in seasonal_aqi.items():
    print(f"   • {season:12s}: {aqi:6.2f} AQI")

# 4. Worst Months
print("\n4️⃣  CRITICAL MONTHS (Highest Pollution)")
print("-" * 80)
monthly_aqi = df.groupby('Month_Name')['AQI'].mean().sort_values(ascending=False).head(3)
for month, aqi in monthly_aqi.items():
    print(f"   • {month:12s}: {aqi:6.2f} AQI")

# 5. Trend Analysis
print("\n5️⃣  TREND ANALYSIS")
print("-" * 80)
recent_years = df[df['Year'] >= df['Year'].max() - 2].groupby('Year')['AQI'].mean()
older_years = df[df['Year'] <= df['Year'].min() + 2].groupby('Year')['AQI'].mean()
trend = "Improving 📉" if recent_years.mean() < older_years.mean() else "Worsening 📈"
print(f"   • Overall Trend: {trend}")
print(f"   • Early Period Avg (first 3 years): {older_years.mean():.2f}")
print(f"   • Recent Period Avg (last 3 years): {recent_years.mean():.2f}")
print(f"   • Change: {((recent_years.mean() - older_years.mean()) / older_years.mean() * 100):+.2f}%")

# 6. Pollutant Contributions
print("\n6️⃣  DOMINANT POLLUTANTS")
print("-" * 80)
print(f"   • SO2 Average: {df['SO2'].mean():.2f} µg/m³")
print(f"   • NOx Average: {df['NOx'].mean():.2f} µg/m³")
print(f"   • RSPM Average: {df['RSPM'].mean():.2f} µg/m³")
dominant = df[['SO2', 'NOx', 'RSPM']].mean().idxmax()
print(f"   • Primary Contributor: {dominant}")

# 7. Key Recommendations
print("\n7️⃣  ACTION RECOMMENDATIONS")
print("-" * 80)
high_pollution_season = seasonal_aqi.index[0]
print(f"   ✓ Focus interventions during {high_pollution_season}")
print(f"   ✓ Implement stricter emission controls in peak months")
print(f"   ✓ Target {dominant} reduction strategies")
if df['Exceeds_Standard'].sum() / len(df) > 0.3:
    print(f"   ✓ URGENT: >30% days exceed acceptable levels - immediate action needed")
print(f"   ✓ Enhance public awareness during high AQI periods")
print(f"   ✓ Promote green transportation and industrial emission controls")

# 8. Monitoring Recommendations
print("\n8️⃣  MONITORING & POLICY RECOMMENDATIONS")
print("-" * 80)
print(f"   ✓ Continue long-term monitoring to track improvement")
print(f"   ✓ Implement real-time air quality alerts for residents")
print(f"   ✓ Develop seasonal action plans for pollution control")
print(f"   ✓ Invest in pollution source identification studies")
print(f"   ✓ Promote urban greening and sustainable practices")

print("\n" + "="*80)

                    📊 NAGPUR AIR QUALITY - DECISION INSIGHTS

1️⃣  OVERALL AIR QUALITY STATUS
--------------------------------------------------------------------------------
   • Average AQI: 62.42
   • Median AQI: 58.00
   • AQI Range: 0.00 - 398.00
   • Data Coverage: 2009-07-27 to 2019-07-18

2️⃣  AQI CATEGORY BREAKDOWN
--------------------------------------------------------------------------------
   • Good           :   750 days (25.22%)
   • Satisfactory   :  1795 days (60.36%)
   • Moderate       :   185 days ( 6.22%)
   • Poor           :     0 days ( 0.00%)
   • Very Poor      :     1 days ( 0.03%)
   • Severe         :     0 days ( 0.00%)

3️⃣  SEASONAL PATTERNS
--------------------------------------------------------------------------------
   • Autumn      :  71.74 AQI
   • Winter      :  69.28 AQI
   • Spring      :  62.38 AQI
   • Monsoon     :  58.37 AQI
   • Summer      :  53.94 AQI

4️⃣  CRITICAL MONTHS (Highest Pollution)
--------------------------------------------

## Step 17: Export Key Metrics for Reporting

In [17]:
# Create summary reports
summary_stats = pd.DataFrame({
    'Metric': ['Average AQI', 'Median AQI', 'Min AQI', 'Max AQI', 'Std Dev AQI',
               'Average SO2', 'Average NOx', 'Average RSPM',
               'Total Days Monitored', 'Days Exceeding Standard (>100)',
               'Percentage Above Standard'],
    'Value': [
        df['AQI'].mean(),
        df['AQI'].median(),
        df['AQI'].min(),
        df['AQI'].max(),
        df['AQI'].std(),
        df['SO2'].mean(),
        df['NOx'].mean(),
        df['RSPM'].mean(),
        len(df),
        df['Exceeds_Standard'].sum(),
        df['Exceeds_Standard'].sum() / len(df) * 100
    ]
})

print("\n📄 SUMMARY STATISTICS TABLE")
print("="*80)
print(summary_stats.to_string(index=False))

# Save to CSV
summary_stats.to_csv('nagpur_aqi_summary.csv', index=False)
yearly_summary = df.groupby('Year')[['SO2', 'NOx', 'RSPM', 'AQI']].mean().round(2)
yearly_summary.to_csv('nagpur_aqi_yearly_summary.csv')

print("\n✅ Summary reports saved:")
print("   • nagpur_aqi_summary.csv")
print("   • nagpur_aqi_yearly_summary.csv")


📄 SUMMARY STATISTICS TABLE
                        Metric       Value
                   Average AQI   62.419993
                    Median AQI   58.000000
                       Min AQI    0.000000
                       Max AQI  398.000000
                   Std Dev AQI   21.838630
                   Average SO2    9.446258
                   Average NOx   28.949663
                  Average RSPM   45.746634
          Total Days Monitored 2974.000000
Days Exceeding Standard (>100)  186.000000
     Percentage Above Standard    6.254203

✅ Summary reports saved:
   • nagpur_aqi_summary.csv
   • nagpur_aqi_yearly_summary.csv
